# Circuit learning module: Lambeq manually with SPSA and JAX

This module performs the optimization of the parametrized circuit manually compared to Lambeq's automatic QuantumTrainer class. I created this because I wanted to have more control over the optimization process and debug it better. The code is based on the workflow presented in https://github.com/CQCL/Quanthoven.

In [1]:
import warnings
import json
import os
import sys
import glob
from math import ceil
from pathlib import Path
from jax import numpy as np
from sympy import default_sort_key
import numpy
import pickle
import matplotlib.pyplot as plt

import jax
from jax import jit
from noisyopt import minimizeSPSA, minimizeCompass

from discopy.quantum import Circuit
from discopy.tensor import Tensor
from discopy.utils import loads
#from pytket.extensions.qiskit import AerBackend
#from pytket.extensions.qulacs import QulacsBackend
#from pytket.extensions.cirq import CirqStateSampleBackend
backend = None

from utils import *
#from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

warnings.filterwarnings('ignore')
this_folder = os.path.abspath(os.getcwd())
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
#os.environ["JAX_PLATFORMS"] = "cpu"

SEED = 0

# This avoids TracerArrayConversionError from jax
Tensor.np = np

rng = numpy.random.default_rng(SEED)
numpy.random.seed(SEED)

## Read circuit data

We read the circuits from the pickled files. Select if we perform binary classification or multi-class classification. Give number of qubits to create classes:
- 1 qubits -> 2^1 = 2 classes i.e. binary classification
- 2 qubits -> 2^2 = 4 classes
- ...
- 5 qubits -> 2^5 = 32 classes, etc.

In [2]:
# Select workload
workload = "execution_time"
#workload = "cardinality"

# Select workload size
#workload_size = "small"
#workload_size = "medium"
#workload_size = "large"
workload_size = "main"

classification = 2
layers = 1
single_qubit_params = 3
n_wire_count = 1

loss = multi_class_loss
acc = multi_class_acc

if classification == 1:
    loss = bin_class_loss
    acc = bin_class_acc

# Access the selected circuits
path_name = this_folder + "//simplified-JOB-diagrams//" + workload + "//" + workload_size + "//circuits//" + str(classification) + "//" + str(layers) + "_layer//" + str(single_qubit_params) + "_single_qubit_params//" + str(n_wire_count) + "_n_wire_count//"

training_circuits_paths = glob.glob(path_name + "training//[0-9]*.p")
validation_circuits_paths = glob.glob(path_name + "validation//[0-9]*.p")
test_circuits_paths = glob.glob(path_name + "test//[0-9]*.p")

In [3]:
training_circuits = read_diagrams(training_circuits_paths)
validation_circuits = read_diagrams(validation_circuits_paths)
test_circuits = read_diagrams(test_circuits_paths)

## Read training and test data

In [4]:
training_data, test_data, validation_data = None, None, None
data_path = this_folder + "//data//" + workload + "//" + workload_size + "//"

with open(data_path + "training_data.json", "r") as inputfile:
    training_data = json.load(inputfile)['training_data']
with open(data_path + "test_data.json", "r") as inputfile:
    test_data = json.load(inputfile)['test_data']
with open(data_path + "validation_data.json", "r") as inputfile:
    validation_data = json.load(inputfile)['validation_data']

training_data_labels = create_labeled_classes(training_data, classification, workload)
test_data_labels = create_labeled_classes(test_data, classification, workload)
validation_data_labels = create_labeled_classes(validation_data, classification, workload)

## Lambeq optimizer

## Model

In [5]:
def make_pred_fn(circuits):
    # In the case we want to use other backends. 
    # Currently does not work properly.
    if backend:
        compiled_circuits1 = backend.get_compiled_circuits([c.to_tk() for c in circuits])
        circuits = [Circuit.from_tk(c) for c in compiled_circuits1]
        
    circuit_fns = [c.lambdify(*parameters) for c in circuits]
    
    def predict(params):
        outputs = Circuit.eval(*(c(*params) for c in circuit_fns), backend = backend)
        res = []
        
        for output in outputs:
            predictions = np.abs(output.array) + 1e-9
            ratio = predictions / predictions.sum()
            res.append(ratio)
            
        return np.array(res)
    return predict

## Loss function and evaluation

In [6]:
def make_cost_fn(pred_fn, labels):
    def cost_fn(params, **kwargs):
        predictions = pred_fn(params)

        cost = loss(predictions, labels) #-np.sum(labels * np.log(predictions)) / len(labels)  # binary cross-entropy loss
        costs.append(cost)

        accuracy = acc(predictions, labels) #np.sum(np.round(predictions) == labels) / len(labels) / 2  # half due to double-counting
        accuracies.append(accuracy)

        return cost

    costs, accuracies = [], []
    return cost_fn, costs, accuracies

## Minimization with noisyopt

In [7]:
def initialize_parameters(old_params, old_values, new_params):
    new_values = list(numpy.array(rng.random(len(new_params))))
    old_param_dict = {}
    for p, v in zip(old_params, old_values):
        old_param_dict[p] = v
        
    parameters = sorted(set(old_params + new_params), key=default_sort_key)
    values = []
    for p in parameters:
        if p in old_param_dict:
            values.append(old_param_dict[p])
        else:
            values.append(new_values.pop())
            
    return parameters, np.array(values)

In [8]:
EPOCHS = 10000
initial_number_of_circuits = 5
syms = {}
limit = False
all_training_keys = list(training_circuits.keys())
initial_circuit_keys = all_training_keys[:initial_number_of_circuits + 1]
current_training_circuits = {}
result_file = workload + "_" + workload_size + "_noisyopt_2_" + str(classification) + "_" + str(layers) + "_" + str(single_qubit_params)

for k in initial_circuit_keys:
    current_training_circuits[k] = training_circuits[k]
    
syms = get_symbols(current_training_circuits)
parameters = sorted(syms, key=default_sort_key)
if initial_number_of_circuits > 20 and os.path.exists("points//" + result_file + ".npz"):
    with open("points//" + result_file + ".npz", "rb") as f:
        print("Loading parameters from file " + result_file)
        npzfile = np.load(f)
        init_params_spsa = npzfile['arr_0']
else:
    print("Initializing new parameters")
    init_params_spsa = np.array(rng.random(len(parameters)))
result = None
run = 0

Initializing new parameters


In [ ]:
result = None

for i, key in enumerate(all_training_keys[initial_number_of_circuits:]):
    print("Progress: ", round((i + initial_number_of_circuits)/len(all_training_keys), 3))
    
    if len(syms) == len(get_symbols(current_training_circuits)) and i > 0:
        if i != len(all_training_keys[1:]):
            current_training_circuits[key] = training_circuits[key]
            new_parameters = sorted(get_symbols({key: training_circuits[key]}), key=default_sort_key)
            if result:
                parameters, init_params_spsa = initialize_parameters(parameters, result.x, new_parameters)
                #continue
            else:
                syms = get_symbols(current_training_circuits)
                parameters = sorted(syms, key=default_sort_key)
                init_params_spsa = np.array(rng.random(len(parameters)))
    else:
        run += 1
    
    # Select those circuits from test and validation circuits which share the parameters with the current training circuits
    current_validation_circuits = select_circuits(current_training_circuits, validation_circuits)
    current_test_circuits = select_circuits(current_training_circuits, test_circuits)
    
    if len(current_validation_circuits) == 0 or len(current_test_circuits) == 0:
        continue
    
    # Create lists with circuits and their corresponding label
    training_circuits_l, training_data_labels_l = construct_data_and_labels(current_training_circuits, training_data_labels)
    validation_circuits_l, validation_data_labels_l = construct_data_and_labels(current_validation_circuits, validation_data_labels)
    test_circuits_l, test_data_labels_l = construct_data_and_labels(current_test_circuits, test_data_labels)
    
    # Limit the number of validation and test circuits to 20% of number of the training circuits
    if limit:
        val_test_circ_size = ceil(len(current_training_circuits))
        if len(current_validation_circuits) > val_test_circ_size:
            validation_circuits_l = validation_circuits_l[:val_test_circ_size]
            validation_data_labels_l = validation_data_labels_l[:val_test_circ_size]
        if len(current_test_circuits) > val_test_circ_size:
            test_circuits_l = test_circuits_l[:val_test_circ_size]
            test_data_labels_l = test_data_labels_l[:val_test_circ_size]
    
    stats = f"Number of training circuits: {len(training_circuits_l)}   "\
        + f"Number of validation circuits: {len(validation_circuits_l)}   "\
        + f"Number of test circuits: {len(test_circuits_l)}   "\
        + f"Number of parameters in model: {len(set([sym for circuit in training_circuits_l for sym in circuit.free_symbols]))}"
    
    with open("results//" + result_file + ".txt", "a") as f:
        f.write(stats + "\n")
    
    print(stats)
    
    optimization_interval = 3
    
    if result == None or run % optimization_interval == 0:
    
        train_pred_fn = jit(make_pred_fn(training_circuits_l))
        dev_pred_fn = jit(make_pred_fn(validation_circuits_l))
        test_pred_fn = make_pred_fn(test_circuits_l)

        train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_fn, training_data_labels_l)
        dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, validation_data_labels_l)

        def callback_fn(xk):
            #print(xk)
            valid_loss = dev_cost_fn(xk)
            train_loss = numpy.around(min(float(train_costs[-1]), float(train_costs[-2])), 4)
            train_acc = numpy.around(min(float(train_accs[-1]), float(train_accs[-2])), 4)
            valid_acc = numpy.around(float(dev_accs[-1]), 4)
            iters = int(len(train_accs)/2)
            if iters % 200 == 0:
                info = f"Epoch: {iters}   "\
                + f"train/loss: {train_loss}   "\
                + f"valid/loss: {numpy.around(float(valid_loss), 4)}   "\
                + f"train/acc: {train_acc}   "\
                + f"valid/acc: {valid_acc}"

                with open("results//" + result_file + ".txt", "a") as f:
                    f.write(info + "\n")

                print(info, file=sys.stderr)
            return valid_loss

        #a_value = 0.0053
        #c_value = 0.0185

        # Good
        #a_value = 0.053
        #c_value = 0.00185

        a_value = 0.0053
        c_value = 0.00185

        train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_fn, training_data_labels_l)
        dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, validation_data_labels_l)

        result = minimizeSPSA(train_cost_fn, x0=init_params_spsa, a = a_value, c = c_value, niter=EPOCHS, callback=callback_fn)
        #result = minimizeCompass(train_cost_fn, x0=init_params_spsa, redfactor=2.0, deltainit=1.0, deltatol=0.001, feps=1e-15, errorcontrol=True, funcNinit=30, funcmultfactor=2.0, paired=True, alpha=0.05, callback=callback_fn)

        figure_path = this_folder + "//results//" + result_file + ".png"
        visualize_result_noisyopt(result, make_cost_fn, test_pred_fn, test_data_labels_l, train_costs, train_accs, dev_costs, dev_accs, figure_path, result_file)
    
    run += 1
    #EPOCHS += 100
    syms = get_symbols(current_training_circuits)
    
    # Extend for the next optimization round
    current_training_circuits[key] = training_circuits[key]
    new_parameters = sorted(get_symbols({key: training_circuits[key]}), key=default_sort_key)
    parameters, init_params_spsa = initialize_parameters(parameters, result.x, new_parameters)

Progress:  0.011
Number of training circuits: 6   Number of validation circuits: 18   Number of test circuits: 19   Number of parameters in model: 130


Epoch: 200   train/loss: 1.2825   valid/loss: 5.8651   train/acc: 0.8333   valid/acc: 0.3889
Epoch: 400   train/loss: 0.9641   valid/loss: 5.4415   train/acc: 0.6667   valid/acc: 0.4444
Epoch: 600   train/loss: 0.834   valid/loss: 5.4546   train/acc: 0.8333   valid/acc: 0.4444
Epoch: 800   train/loss: 0.7837   valid/loss: 5.7005   train/acc: 0.8333   valid/acc: 0.4444
Epoch: 1000   train/loss: 0.7007   valid/loss: 5.7869   train/acc: 0.8333   valid/acc: 0.4444
Epoch: 1200   train/loss: 0.6124   valid/loss: 5.6694   train/acc: 0.8333   valid/acc: 0.4444
Epoch: 1400   train/loss: 0.5415   valid/loss: 5.7728   train/acc: 0.8333   valid/acc: 0.3889
Epoch: 1600   train/loss: 0.4852   valid/loss: 6.4443   train/acc: 0.8333   valid/acc: 0.3889
Epoch: 1800   train/loss: 0.449   valid/loss: 6.2649   train/acc: 0.8333   valid/acc: 0.3889
Epoch: 2000   train/loss: 0.397   valid/loss: 6.338   train/acc: 0.8333   valid/acc: 0.3889
Epoch: 2200   train/loss: 0.3558   valid/loss: 6.4066   train/acc: 0

Test accuracy: 0.05263157894736842
Progress:  0.013
Number of training circuits: 7   Number of validation circuits: 22   Number of test circuits: 25   Number of parameters in model: 133
Progress:  0.016
Number of training circuits: 8   Number of validation circuits: 29   Number of test circuits: 30   Number of parameters in model: 136


Epoch: 200   train/loss: 1.7126   valid/loss: 11.7808   train/acc: 0.625   valid/acc: 0.1034
Epoch: 400   train/loss: 1.5177   valid/loss: 12.6756   train/acc: 0.625   valid/acc: 0.1724
Epoch: 600   train/loss: 1.3557   valid/loss: 13.052   train/acc: 0.75   valid/acc: 0.1724
Epoch: 800   train/loss: 1.3121   valid/loss: 13.1225   train/acc: 0.875   valid/acc: 0.2069
Epoch: 1000   train/loss: 1.1987   valid/loss: 13.1693   train/acc: 0.875   valid/acc: 0.2759
Epoch: 1200   train/loss: 1.1372   valid/loss: 13.584   train/acc: 0.875   valid/acc: 0.3103
Epoch: 1400   train/loss: 1.0449   valid/loss: 12.9183   train/acc: 0.875   valid/acc: 0.3103
Epoch: 1600   train/loss: 0.9792   valid/loss: 12.9432   train/acc: 0.875   valid/acc: 0.3103
Epoch: 1800   train/loss: 0.9198   valid/loss: 12.8172   train/acc: 0.875   valid/acc: 0.2759
Epoch: 2000   train/loss: 0.9041   valid/loss: 13.1957   train/acc: 0.875   valid/acc: 0.2414
Epoch: 2200   train/loss: 0.8317   valid/loss: 13.6766   train/acc:

Test accuracy: 0.23333333333333334
Progress:  0.018
Number of training circuits: 9   Number of validation circuits: 37   Number of test circuits: 35   Number of parameters in model: 148
Progress:  0.02
Number of training circuits: 10   Number of validation circuits: 45   Number of test circuits: 42   Number of parameters in model: 157
Progress:  0.022
Number of training circuits: 11   Number of validation circuits: 47   Number of test circuits: 46   Number of parameters in model: 169


...

Test accuracy: 0.4864864864864865
Progress:  0.741
Number of training circuits: 331   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267
Progress:  0.743
Number of training circuits: 332   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267
Progress:  0.746
Number of training circuits: 333   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267


Epoch: 200   train/loss: 115.1691   valid/loss: 39.0528   train/acc: 0.3423   valid/acc: 0.292
Epoch: 400   train/loss: 113.0751   valid/loss: 37.5548   train/acc: 0.2492   valid/acc: 0.3274
Epoch: 600   train/loss: 110.7318   valid/loss: 39.2888   train/acc: 0.2222   valid/acc: 0.1416
Epoch: 800   train/loss: 111.6726   valid/loss: 38.8969   train/acc: 0.3784   valid/acc: 0.2832
Epoch: 1000   train/loss: 109.3552   valid/loss: 39.3108   train/acc: 0.3604   valid/acc: 0.2478
Epoch: 1200   train/loss: 107.1911   valid/loss: 37.797   train/acc: 0.3724   valid/acc: 0.3186
Epoch: 1400   train/loss: 105.5237   valid/loss: 39.8084   train/acc: 0.4084   valid/acc: 0.2832
Epoch: 1600   train/loss: 103.7532   valid/loss: 40.0985   train/acc: 0.4685   valid/acc: 0.292
Epoch: 1800   train/loss: 101.2943   valid/loss: 39.911   train/acc: 0.4535   valid/acc: 0.3009
Epoch: 2000   train/loss: 101.1664   valid/loss: 38.9456   train/acc: 0.4865   valid/acc: 0.3628
Epoch: 2200   train/loss: 101.7469   v

Test accuracy: 0.4144144144144144
Progress:  0.748
Number of training circuits: 334   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267
Progress:  0.75
Number of training circuits: 335   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267
Progress:  0.752
Number of training circuits: 336   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267


Epoch: 200   train/loss: 114.0464   valid/loss: 40.3044   train/acc: 0.3542   valid/acc: 0.2478
Epoch: 400   train/loss: 116.2649   valid/loss: 39.8257   train/acc: 0.3065   valid/acc: 0.2566
Epoch: 600   train/loss: 115.4419   valid/loss: 39.4044   train/acc: 0.2649   valid/acc: 0.2655
Epoch: 800   train/loss: 112.8329   valid/loss: 39.2953   train/acc: 0.3214   valid/acc: 0.2655
Epoch: 1000   train/loss: 111.5548   valid/loss: 39.4473   train/acc: 0.3452   valid/acc: 0.3097
Epoch: 1200   train/loss: 111.9532   valid/loss: 40.1398   train/acc: 0.3542   valid/acc: 0.3186
Epoch: 1400   train/loss: 108.7278   valid/loss: 39.1874   train/acc: 0.3423   valid/acc: 0.3009
Epoch: 1600   train/loss: 112.5316   valid/loss: 43.8036   train/acc: 0.3304   valid/acc: 0.1504
Epoch: 1800   train/loss: 108.9566   valid/loss: 40.3713   train/acc: 0.3036   valid/acc: 0.2655
Epoch: 2000   train/loss: 113.0739   valid/loss: 40.0306   train/acc: 0.253   valid/acc: 0.1947
Epoch: 2200   train/loss: 115.4847 

Test accuracy: 0.44144144144144143
Progress:  0.754
Number of training circuits: 337   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267
Progress:  0.757
Number of training circuits: 338   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267
Progress:  0.759
Number of training circuits: 339   Number of validation circuits: 113   Number of test circuits: 111   Number of parameters in model: 267
